### Starting up the Docker container
Simply download docker from the [docker website](https://www.docker.com/get-started) and run the following command in the terminal:
`docker run -it -p 8888:8888 jupyter/pyspark-notebook`

### Mounting the host volume
- If you want to store your created notebook in you local machine you need to mount your local directory to the docker container
- TL;DR TO mount `~/dockerhome` in your machine to `/home/jovyan`(absolute path needed) inside your docker container you can add `-v ~/dockerhome:/home/jovyan` option in above command
- Final command `docker run -it -p 8888:8888 -v ~/dockerhome:/home/jovyan jupyter/pyspark-notebook`
- https://www.digitalocean.com/community/tutorials/how-to-share-data-between-the-docker-container-and-the-host

### Original docker image
- https://hub.docker.com/r/jupyter/pyspark-notebook/

### Download Data

In [2]:
# Data can also be downloaded from here: https://data.vermont.gov/Finance/Vermont-Vendor-Payments/786x-sbp3
# ! curl -o VermontVendor.csv https://data.vermont.gov/api/views/786x-sbp3/rows.csv?accessType=DOWNLOAD 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  284M    0  284M    0     0  3642k      0 --:--:--  0:01:19 --:--:-- 4926k


### Setting up the Spark session

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession

/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
spark = (
    SparkSession
    .builder
    .appName('getting-started-with-pyspark')
    .getOrCreate()
)

In [14]:
%%timeit # Reading in CSV
df = spark.read.csv('VermontVendor.csv', header=True)

463 ms ± 50.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit # REadiang CSV with pandas for time comparison
df = pd.read_csv('VermontVendor.csv')

7.71 s ± 1.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
# Reading CSV again for analysis
df = spark.read.csv('VermontVendor.csv', header=True, inferSchema=True)
# Casting `Amount` to double
df = df.withColumn("Amount", df["Amount"].cast("double"))

### Basic Spark Methods

In [19]:
df.show(1)

+--------------+--------------------+------+-------------+--------------------+----+-----+------------------+----------+------+--------------------+------+--------------------+-----+
|Quarter Ending|          Department|UnitNo|Vendor Number|              Vendor|City|State|DeptID Description|    DeptID|Amount|             Account|AcctNo|    Fund Description| Fund|
+--------------+--------------------+------+-------------+--------------------+----+-----+------------------+----------+------+--------------------+------+--------------------+-----+
|    09/30/2009|Environmental Con...|  6140|   0000276016|1st Run Computer ...|null|   NY|   WQD - Waterbury|6140040206| 930.0|Rep&Maint-Info Te...|513000|Environmental Per...|21295|
+--------------+--------------------+------+-------------+--------------------+----+-----+------------------+----------+------+--------------------+------+--------------------+-----+
only showing top 1 row



In [20]:
df.head(1)

[Row(Quarter Ending='09/30/2009', Department='Environmental Conservation', UnitNo=6140, Vendor Number='0000276016', Vendor='1st Run Computer Services Inc', City=None, State='NY', DeptID Description='WQD - Waterbury', DeptID='6140040206', Amount=930.0, Account='Rep&Maint-Info Tech Hardware', AcctNo='513000', Fund Description='Environmental Permit Fund', Fund='21295')]

In [18]:
df.columns

['Quarter Ending',
 'Department',
 'UnitNo',
 'Vendor Number',
 'Vendor',
 'City',
 'State',
 'DeptID Description',
 'DeptID',
 'Amount',
 'Account',
 'AcctNo',
 'Fund Description',
 'Fund']

In [30]:
df.describe().show()

+-------+--------------+--------------------+------------------+------------------+--------------------+--------+------------------+------------------+--------------------+------------------+-------------------+------------------+--------------------+--------------------+
|summary|Quarter Ending|          Department|            UnitNo|     Vendor Number|              Vendor|    City|             State|DeptID Description|              DeptID|            Amount|            Account|            AcctNo|    Fund Description|                Fund|
+-------+--------------+--------------------+------------------+------------------+--------------------+--------+------------------+------------------+--------------------+------------------+-------------------+------------------+--------------------+--------------------+
|  count|       1648466|             1648466|           1648466|           1648466|             1648466|  906143|           1648418|           1647929|             1648466|         

### Querying the Data

The following code registers temporary table and selects a few columns using SQL syntax:

In [ ]:
df.createOrReplaceTempView('VermontVendor')

### Spark SELECT

In [32]:
spark.sql(
'''
SELECT `Quarter Ending`, Department, Amount, State FROM VermontVendor
LIMIT 10
'''
).show()

+--------------+--------------------+------+-----+
|Quarter Ending|          Department|Amount|State|
+--------------+--------------------+------+-----+
|    09/30/2009|Environmental Con...| 930.0|   NY|
|    09/30/2009|Environmental Con...| 930.0|   NY|
|    09/30/2009|Vermont Veterans'...|  24.0|   CT|
|    09/30/2009|Vermont Veterans'...| 420.0|   CT|
|    09/30/2009|         Corrections| 270.8|   PA|
|    09/30/2009|         Corrections|  35.0|   PA|
|    09/30/2009|       Public Safety| 971.4|   PA|
|    09/30/2009|Agriculture, Food...| 60.59|   TX|
|    09/30/2009|Agriculture, Food...|541.62|   TX|
|    09/30/2009|              Health|283.98|   PA|
+--------------+--------------------+------+-----+



#### Above method is same as below line

In [33]:
df.select('Quarter Ending', 'Department', 'Amount', 'State').show(10)

+--------------+--------------------+------+-----+
|Quarter Ending|          Department|Amount|State|
+--------------+--------------------+------+-----+
|    09/30/2009|Environmental Con...| 930.0|   NY|
|    09/30/2009|Environmental Con...| 930.0|   NY|
|    09/30/2009|Vermont Veterans'...|  24.0|   CT|
|    09/30/2009|Vermont Veterans'...| 420.0|   CT|
|    09/30/2009|         Corrections| 270.8|   PA|
|    09/30/2009|         Corrections|  35.0|   PA|
|    09/30/2009|       Public Safety| 971.4|   PA|
|    09/30/2009|Agriculture, Food...| 60.59|   TX|
|    09/30/2009|Agriculture, Food...|541.62|   TX|
|    09/30/2009|              Health|283.98|   PA|
+--------------+--------------------+------+-----+
only showing top 10 rows



### Spark WHERE

In [34]:
spark.sql(
'''
SELECT `Quarter Ending`, Department, Amount, State FROM VermontVendor 
WHERE Department = 'Education'
LIMIT 10
'''
).show()

+--------------+----------+-------+-----+
|Quarter Ending|Department| Amount|State|
+--------------+----------+-------+-----+
|    09/30/2009| Education|9423.36|   VT|
|    09/30/2009| Education| 110.03|   IL|
|    09/30/2009| Education| 332.58|   IL|
|    09/30/2009| Education|  60.08|   IL|
|    09/30/2009| Education| 284.83|   IL|
|    09/30/2009| Education| 377.15|   IL|
|    09/30/2009| Education| 114.74|   IL|
|    09/30/2009| Education| 129.72|   IL|
|    09/30/2009| Education| 114.54|   IL|
|    09/30/2009| Education|  375.6|   IL|
+--------------+----------+-------+-----+



#### Same as below

In [35]:
df.select('Quarter Ending', 'Department', 'Amount', 'State').filter(df['Department'] == 'Education').show(10)

+--------------+----------+-------+-----+
|Quarter Ending|Department| Amount|State|
+--------------+----------+-------+-----+
|    09/30/2009| Education|9423.36|   VT|
|    09/30/2009| Education| 110.03|   IL|
|    09/30/2009| Education| 332.58|   IL|
|    09/30/2009| Education|  60.08|   IL|
|    09/30/2009| Education| 284.83|   IL|
|    09/30/2009| Education| 377.15|   IL|
|    09/30/2009| Education| 114.74|   IL|
|    09/30/2009| Education| 129.72|   IL|
|    09/30/2009| Education| 114.54|   IL|
|    09/30/2009| Education|  375.6|   IL|
+--------------+----------+-------+-----+
only showing top 10 rows



### That's it folks

#### Reference
- https://levelup.gitconnected.com/using-docker-and-pyspark-134cd4cab867